In [1]:
import numpy as np
import optuna
from optuna import Trial
import optunahub

from evo import evo_alg
from shpm2 import SHPM

import random

from copy import deepcopy
import matplotlib.pyplot as plt

/Users/lachlanstewart/anaconda3/envs/dspy311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from optuna.visualization import plot_parallel_coordinate, plot_param_importances, plot_contour, plot_slice

In [3]:
random.seed(42)

In [ ]:
# consistent parameters
n = 10

population_size = 100
parents = 5
max_iters = 50

In [5]:
shpm = SHPM(n)

In [6]:
# targets:
n_targets = 10

P_targets = [
    shpm.random_point() for i in range(n_targets)
]

In [7]:
A_targets = [
    shpm.assemble(P) for P in P_targets
]

In [8]:
initial_population = [
    shpm.random_point() for i in range(population_size)
]

In [9]:
def trial_objective(trial: Trial):

    # suggest parameter values            
    params = {
        ## mutation magnitudes
        "Q_mut_mag": trial.suggest_float("Q_mut_mag", 1e-4, 1e-1, log=True),
        "UT_mut_mag": trial.suggest_float("UT_mut_mag", 1e-4, 1e-1, log=True),
        "SD_mut_mag": trial.suggest_float("SD_mut_mag", 1e-4, 1e-1, log=True),
        "HD_mut_mag": trial.suggest_float("HD_mut_mag", 1e-4, 1e-1, log=True),
        ## event probabilities
        "Q_swap_chance": trial.suggest_float("Q_swap_chance", 1e-4, 1e-1, log=False),
        "crazy_Q_xv_chance": trial.suggest_float("crazy_Q_xv_chance", 1e-2, 2e-1, log=False),
        "HD_1x1_merge_chance": trial.suggest_float("HD_1x1_merge_chance", 1e-2, 5e-1, log=False),
        "HD_2x2_split_chance": trial.suggest_float("HD_2x2_split_chance", 1e-2, 5e-1, log=False),
        "HD_eig_swap_chance": trial.suggest_float("HD_eig_swap_chance", 1e-2, 5e-1, log=False)    
    }
    crossover_chance = trial.suggest_float("crossover_chance", 1e-1, 6e-1, log=True)

    shpm_trial = SHPM(n, **params)

    best_objectives = []

    for target_idx, A_target in enumerate(A_targets):

        A_target_norm = np.linalg.norm(A_target)

        def evo_objective(P):            
            
            A = shpm.assemble(P)
            distance = np.linalg.norm(A - A_target) / A_target_norm
            return distance

        _, _, _, population_objectives = evo_alg(
            deepcopy(initial_population),
            parents, population_size,
            evo_objective,
            shpm.mutate, shpm.crossover,
            s=1.0, c=crossover_chance,
            maxiter=max_iters, eps=1e-5
        )

        best_objectives.append(population_objectives[-1][0])
    
    best_obj_avg = np.sum(best_objectives) / n_targets

    return best_obj_avg

In [10]:
module = optunahub.load_module(package="samplers/auto_sampler")
study = optuna.create_study(sampler=module.AutoSampler())

[I 2025-05-16 14:17:03,831] A new study created in memory with name: no-name-d49e6926-9816-466b-8e0e-08d2861c601a


In [11]:
study.optimize(
    trial_objective,
    48,
    n_jobs=8,
    show_progress_bar=True    
)

Best trial: 2. Best value: 0.27706:   2%|▏         | 1/48 [00:51<40:43, 51.99s/it]/Users/lachlanstewart/.cache/optunahub/api.github.com/optuna/optunahub-registry/main/package/samplers/auto_sampler/_sampler.py:184: ExperimentalWarning: GPSampler is experimental (supported from v3.6.0). The interface can change in the future.
  return GPSampler(seed=seed)


[I 2025-05-16 14:17:55,890] Trial 2 finished with value: 0.27706026560695673 and parameters: {'Q_mut_mag': 0.0015623162513037315, 'UT_mut_mag': 0.01982091966460146, 'SD_mut_mag': 0.018689928910389, 'HD_mut_mag': 0.0009884000545812963, 'Q_swap_chance': 0.04190703775692623, 'crazy_Q_xv_chance': 0.10893662237588315, 'HD_1x1_merge_chance': 0.20615607600707925, 'HD_2x2_split_chance': 0.1274011137751831, 'HD_eig_swap_chance': 0.11634866454895675, 'crossover_chance': 0.5632091846702814}. Best is trial 2 with value: 0.27706026560695673.


Best trial: 1. Best value: 0.262483:   4%|▍         | 2/48 [00:56<18:19, 23.89s/it]

[I 2025-05-16 14:18:00,119] Trial 1 finished with value: 0.26248280979704697 and parameters: {'Q_mut_mag': 0.00043725172809726346, 'UT_mut_mag': 0.0006223961455263066, 'SD_mut_mag': 0.056531983605177555, 'HD_mut_mag': 0.0013477878032027361, 'Q_swap_chance': 0.07023395247018586, 'crazy_Q_xv_chance': 0.12597992647060383, 'HD_1x1_merge_chance': 0.3514576430707528, 'HD_2x2_split_chance': 0.4935825106024998, 'HD_eig_swap_chance': 0.4417462595276038, 'crossover_chance': 0.49056584139884796}. Best is trial 1 with value: 0.26248280979704697.


Best trial: 0. Best value: 0.233644:   6%|▋         | 3/48 [01:06<13:06, 17.47s/it]

...

[I 2025-05-16 14:25:43,812] Trial 44 finished with value: 0.23999921834631155 and parameters: {'Q_mut_mag': 0.1, 'UT_mut_mag': 0.1, 'SD_mut_mag': 0.0007125953962459208, 'HD_mut_mag': 0.030789877910842978, 'Q_swap_chance': 0.0001, 'crazy_Q_xv_chance': 0.09805908848933735, 'HD_1x1_merge_chance': 0.01, 'HD_2x2_split_chance': 0.01, 'HD_eig_swap_chance': 0.4576752978690396, 'crossover_chance': 0.11278319299818185}. Best is trial 30 with value: 0.20189938632938792.


Best trial: 30. Best value: 0.201899:  96%|█████████▌| 46/48 [08:44<00:14,  7.19s/it]

[I 2025-05-16 14:25:48,773] Trial 45 finished with value: 0.2254100738398097 and parameters: {'Q_mut_mag': 0.1, 'UT_mut_mag': 0.1, 'SD_mut_mag': 0.0010439702044888696, 'HD_mut_mag': 0.03450443274960491, 'Q_swap_chance': 0.0001, 'crazy_Q_xv_chance': 0.09955233858879277, 'HD_1x1_merge_chance': 0.01, 'HD_2x2_split_chance': 0.01, 'HD_eig_swap_chance': 0.48224089135570114, 'crossover_chance': 0.11747177509115567}. Best is trial 30 with value: 0.20189938632938792.


Best trial: 30. Best value: 0.201899: 100%|██████████| 48/48 [08:47<00:00, 10.99s/it]

[I 2025-05-16 14:25:51,115] Trial 46 finished with value: 0.2292925595946073 and parameters: {'Q_mut_mag': 0.1, 'UT_mut_mag': 0.09999999999999998, 'SD_mut_mag': 0.1, 'HD_mut_mag': 0.09999999999999998, 'Q_swap_chance': 0.0001, 'crazy_Q_xv_chance': 0.01, 'HD_1x1_merge_chance': 0.01, 'HD_2x2_split_chance': 0.2923331150897273, 'HD_eig_swap_chance': 0.2597040135453182, 'crossover_chance': 0.10000000000000002}. Best is trial 30 with value: 0.20189938632938792.
[I 2025-05-16 14:25:51,264] Trial 47 finished with value: 0.2076516296352823 and parameters: {'Q_mut_mag': 0.1, 'UT_mut_mag': 0.1, 'SD_mut_mag': 0.1, 'HD_mut_mag': 0.1, 'Q_swap_chance': 0.0001, 'crazy_Q_xv_chance': 0.01, 'HD_1x1_merge_chance': 0.01, 'HD_2x2_split_chance': 0.2812076327800327, 'HD_eig_swap_chance': 0.2392080435413522, 'crossover_chance': 0.1128275024494426}. Best is trial 30 with value: 0.20189938632938792.


In [12]:
study.best_params

{'Q_mut_mag': 0.04238724079580784,
 'UT_mut_mag': 0.1,
 'SD_mut_mag': 0.0016495791187759506,
 'HD_mut_mag': 0.002322270394265561,
 'Q_swap_chance': 0.0001,
 'crazy_Q_xv_chance': 0.1080669103082098,
 'HD_1x1_merge_chance': 0.01,
 'HD_2x2_split_chance': 0.3913867533195319,
 'HD_eig_swap_chance': 0.38655383320923464,
 'crossover_chance': 0.1468254018385694}

In [13]:
# ax = plot_parallel_coordinate(study)
# ax.figure.set_size_inches((15, 5))
# plt.show()
plot_parallel_coordinate(study)

In [14]:
plot_contour(study, params=["crossover_chance", "crazy_Q_xv_chance", "Q_swap_chance"])

In [15]:
plot_param_importances(study)

In [16]:
plot_slice(study)